# Evidencia 1. Actividad Integradora
## Parte 1. Sistemas multiagentes

Ruy Guzmán Camacho | A01639912

Adrián Becerra Meza | A01639813

Santiago González de Cosío Romero | A01640329

#### Librerías


In [17]:
#model design
import agentpy as ap

#visualizations
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import IPython
import random
import time




### Definición de modelo

In [18]:
pos = [1, 2, 3, 4]
moves = [(0,-1), (1, 0)]

#Controls

UP = (1,0)
DOWN = (-1,0)
RIGHT = (0,1)
LEFT = (0,-1)
DOWNRIGHT = (-1,1)
DOWNLEFT = (-1,-1)

MOVES = [UP, DOWN, RIGHT, LEFT, DOWNRIGHT, DOWNLEFT]

class StreetModel(ap.Model):

    def makeCircle(self,startX,startY,r):
        map = []
        Epsilon = 2.2
        for y in range(startX-r,startX+r+1):
            for x in range(startY-r,startY+r+1):
                # see if we're close to (x-a)**2 + (y-b)**2 == r**2
                if abs((x-startX)**2 + (y-startY)**2 - r**2) < Epsilon**2:
                    map.append((x,y))
        return map
        
    def setup(self):

        # auxiliar variables for positioning cars
        self.h_count = 0
        self.v_count = 0
        self.flag = 0

        self.traverse = []

        # Create agents (trees)
        n_trees = int(self.p['Tree density'] * (self.p.size**2))
        trees = self.agents = ap.AgentList(self, n_trees)
        self.carros = ap.AgentList(self, self.p['trafico'])

        # Create grid (street)
        self.street = ap.Grid(self, [self.p.size]*2, track_empty=True)
        #self.street.add_agents(trees, random=True, empty=True)
        self.street.add_agents(self.carros,[(2,21)]*self.p['trafico'])
        #self.street.add_agents(trees, [(0,21),(13,21)])

        # Initiate a dynamic variable for all elements
        # Condition 0: trees, 1: car, 2: street 
        # position 0: top, 1: left
        # Draw trees

        #Initial position of number of cars available for traffic simulation
        pos_count = self.p.size -4
        self.carros.position = 0
        self.carros.inside = 0
        for carro in self.carros:
            carro.position = random.choice(pos)
            self.street.move_to(carro, (37, pos_count))
            pos_count -= 10
        
        self.agents.condition = 0
    
        # Draw Street
        #outerPoints = self.makeCircle(int(self.p.size/2),int(self.p.size/2),8)
        #self.outerCrossroad = ap.AgentList(self,len(outerPoints))
       

        innerPoints = self.makeCircle(int(self.p.size/2),int(self.p.size/2), 7)
        self.road = innerPoints
        self.innerCrossroad = ap.AgentList(self,len(innerPoints)*2)

        #self.street.add_agents(self.outerCrossroad,outerPoints)
        self.street.add_agents(self.innerCrossroad,innerPoints)
        
        #self.outerCrossroad.condition = 2
        self.innerCrossroad.condition = 0
        self.innerCrossroad.exit = 0
        self.innerCrossroad[37].exit = 1

        #street1 = self.street.agents[self.circlePoints]
        #street2 = self.street.agents[int(self.p.size/8)*3:int(self.p.size/8)*5, int(self.p.size/8)*3:int(self.p.size/8)*5]
        #street1.condition = 2
        #street2.condition = 2

        # draw cars
        self.carros.condition = 1
        # position cars
        """for carro in self.carros:
            carro.position = random.choice(pos)
            if carro.position == 0:
                self.h_count += 1
                #horizontal
                self.street.move_to(carro, (int(self.p.size*0.5),2))
            else:
                #vertical
                self.v_count += 1
                self.street.move_to(carro, (2,int(self.p.size*0.5)))"""



    def step(self):

        # Select cars
        carros = self.carros
        listcars = list(carros)
        road = self.road
        positions = self.street.positions
        
        startPoint = (37, self.p.size -3)
        #tree = self.agents.select(self.agents.condition == 0)

        #para que siga la ruta
        

        # move cars depending on position
        for carro in carros:
            
            #Remaining cars movement until StartPosition into simulation
            if (positions[carro]) != startPoint and carro.inside == 0:
                self.street.move_to(carro,(positions[carro][0], positions[carro][1] + 1))
                if (positions[carro]) == startPoint:
                    carro.inside = 1
                    #Car spawn in correct position
                    if carro.position == 1:
                        self.street.move_to(carro,(2,21))
                    elif carro.position == 2:
                        self.street.move_to(carro, (21,2))
                    elif carro.position == 3:
                        self.street.move_to(carro, (35, 21))
                    elif carro.position == 4:
                        self.street.move_to(carro, (21,37))        
            
            else:
                
                #print(positions[listcars[0]])
                ActPos = positions[carro]

                neighbors = self.street.neighbors(carro)
                listNeighbor = list(neighbors)
                listNeighbor.reverse()

                if listNeighbor:
                    for neighbor in listNeighbor:
                        
                        if neighbor.condition == 0:
                            if neighbor.exit == 1:
                                self.street.move_to(carro,(2,21))
                                self.traverse[0].condition = 0
                                self.traverse.pop(0)
                                break
                            else:
                                #print(neighbor)
                                self.street.move_to(carro, positions[neighbor])
                                neighbor.condition = 1
                                self.traverse.append(neighbor) 

                    self.traverse[0].condition = 0
                    self.traverse.pop(0)

                #bajara hasta que se tope con la glorieta                
                else:
                    if carro.position == 1:
                        self.street.move_by(carro, (1,0))
                    elif carro.position == 2:
                        self.street.move_by(carro, (0,1))
                    elif carro.position == 3:
                        self.street.move_by(carro, (-1,0))
                    elif carro.position == 4:
                        self.street.move_by(carro, (0,-1))
                
    

            burning_trees = self.agents.select(self.agents.condition == 0)
            if len(burning_trees) == 0:
                self.stop()


    def end(self):
        
        # Document a measure at the end of the simulation
        burned_trees = len(self.agents.select(self.agents.condition == 2))
        self.report('Percentage of burned trees',
                    burned_trees / len(self.agents))
                    

### Parametros

In [19]:
# Define parameters

parameters = {
    'Tree density': 1, # Percentage of grid covered by trees
    'size': 40,
    'n': 20, # Height and length of the grid
    'steps': 70,
    'trafico': 4
    }


### Renderizado

In [20]:
# Create single-run animation with custom colors

def animation_plot(model, ax):
    attr_grid = model.street.attr_grid('condition')
    color_dict = {0:'#7FC97F', 1:'#d62c2c', 2:'#808080', None:'#d5e5d5'}
    ap.gridplot(attr_grid, ax=ax, color_dict=color_dict, convert=True)
    ax.set_title(f"Simulation of a forest fire\n"
                 f"Time-step: {model.t}, Trees left: "
                 f"{len(model.agents.select(model.agents.condition == 0))}")
    

fig, ax = plt.subplots()

circle1 = plt.Circle((20,20), 10, color='r')

ax.add_patch(circle1)
model = StreetModel(parameters)
animation = ap.animate(model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=50))